In [1]:
import os
os.environ["SPS_HOME"] = '/global/homes/e/eramey16/fsps/'

import fsps
import dynesty
import sedpy
import h5py, astropy
import numpy as np
import pandas as pd
from docker import classify

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [2]:
ls_id = 9906619143751091 #IPTF16geu

In [3]:
from prospect import prospect_args
parser = prospect_args.get_parser()
parser.print_help()

usage: ipykernel_launcher.py [-h] [--verbose VERBOSE] [--debug]
                             [--outfile OUTFILE] [--output_pickle]
                             [--zcontinuous ZCONTINUOUS] [--optimize]
                             [--min_method MIN_METHOD] [--min_opts MIN_OPTS]
                             [--nmin NMIN] [--emcee] [--nwalkers NWALKERS]
                             [--niter NITER] [--nburn [NBURN ...]]
                             [--save_interval INTERVAL]
                             [--restart_from RESTART_FROM]
                             [--ensemble_dispersion INITIAL_DISP] [--dynesty]
                             [--nested_bound {single,multi,balls,cubes}]
                             [--nested_sample {unif,rwalk,slice}]
                             [--nested_walks NESTED_WALKS]
                             [--nlive_init NESTED_NLIVE_INIT]
                             [--nlive_batch NESTED_NLIVE_BATCH]
                             [--nested_dlogz_init NESTED_DLOGZ_

In [4]:
# Try it with a more standard fit
from sedpy.observate import load_filters
from prospect.utils.obsutils import fix_obs

bkdata, tbldata = classify.get_galaxy(ls_id=ls_id)
gal_data = tbldata.iloc[0]
gal_data

ls_id            9906619143751091
ra                     316.066216
dec                     -6.340221
type                          REX
dered_mag_g             20.250755
dered_mag_r              18.86748
dered_mag_z             17.944523
dered_mag_w1             17.25038
dered_mag_w2            17.611616
dered_flux_g             7.937765
dered_flux_r             28.37973
dered_flux_z             66.40348
dered_flux_w1           125.84846
dered_flux_w2            90.23046
snr_g                   128.86171
snr_r                   213.08531
snr_z                   480.05804
snr_w1                  143.01906
snr_w2                   59.99879
flux_ivar_g              430.3113
flux_ivar_r              78.43762
flux_ivar_z             62.868927
flux_ivar_w1             1.328257
flux_ivar_w2             0.449847
dchisq_1                236241.66
dchisq_2                263783.94
dchisq_3                266028.38
dchisq_4                264630.44
dchisq_5                      0.0
rchisq_g      

In [5]:
# Get galaxy data
bands = ['g', 'r', 'z', 'w1', 'w2']
sdss = ['sdss_{}0'.format(b) for b in ['g','r','z']]
wise = ['wise_w1', 'wise_w2']
filternames = sdss + wise

filters = load_filters(filternames)
maggies = np.array([10**(-.4*gal_data[f'dered_mag_{b}']) for b in bands])
magerr = [ 2.5 / (np.log(10) * gal_data['dered_flux_'+b] * 
                        np.sqrt(gal_data['flux_ivar_'+b])) for b in bands]
magerr = np.clip(magerr, 0.05, np.inf)

In [6]:
# Build obs
obs = dict(wavelength=None, spectrum=None, unc=None, redshift=gal_data['z_phot_median'],
           maggies=maggies, maggies_unc=magerr * maggies / 1.086, filters=filters)
obs = fix_obs(obs)

In [7]:
# Build model
from prospect.models.templates import TemplateLibrary
from prospect.models import PolySpecModel
model_params = TemplateLibrary["continuity_sfh"]
model_params.update(TemplateLibrary["nebular"])
model_params["zred"]["init"] = obs["redshift"]

model = PolySpecModel(model_params)
noise_model = (None, None)

print(model)

:::::::
<class 'prospect.models.sedmodel.PolySpecModel'>

Free Parameters: (name: prior) 
-----------
  logzsol: <class 'prospect.models.priors.TopHat'>(mini=-2,maxi=0.19)
  dust2: <class 'prospect.models.priors.TopHat'>(mini=0.0,maxi=2.0)
  logmass: <class 'prospect.models.priors.TopHat'>(mini=7,maxi=12)
  logsfr_ratios: <class 'prospect.models.priors.StudentT'>(mean=[0. 0.],scale=[0.3 0.3],df=[2 2])

Fixed Parameters: (name: value [, depends_on]) 
-----------
  zred: [0.263183] 
  mass: [1000000.] <function logsfr_ratios_to_masses at 0x7f57687b7f70>
  sfh: [3] 
  imf_type: [2] 
  dust_type: [0] 
  agebins: [[ 0.  8.]
 [ 8.  9.]
 [ 9. 10.]] 
  add_neb_emission: [ True] 
  add_neb_continuum: [ True] 
  nebemlineinspec: [ True] 
  gas_logz: [0.] <function stellar_logzsol at 0x7f5768d96310>
  gas_logu: [-2.] 


In [8]:
from prospect.sources import CSPSpecBasis
sps = CSPSpecBasis(zcontinuous=1)
print(sps.ssp.libraries)

(b'mist', b'miles', b'DL07')


In [ ]:
from prospect.fitting import lnprobfn, fit_model
fitting_kwargs = dict(nlive_init=400, nested_method="rwalk", nested_target_n_effective=1000, nested_dlogz_init=0.05)
output = fit_model(obs, model, sps, optimize=True, dynesty=True, lnprobfn=lnprobfn, noise=noise_model, **fitting_kwargs)
result, duration = output["sampling"]

In [ ]:
from prospect.io import write_results as writer
hfile = "./quickstart_dynesty_mcmc.h5"
writer.write_hdf5(hfile, {}, model, obs,
                 output["sampling"][0], None,
                 sps=sps,
                 tsample=output["sampling"][1],
                 toptimize=0.0)